This notebook is a proto for the ETL process which will be complete in etl.py

In [ ]:
import psycopg2
import os
import glob
import pandas as pd
from sql_queries import * 

In [ ]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=admin")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get cursor to the Database")
    print(e)
conn.set_session(autocommit=True)

In [ ]:
# the following funtion lists all files in a given filepath
def get_files(filepath):
    all_files=[]
    for root, dirs, files in os.walk(filepath):
        files=glob.glob(os.path.join(root,'*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))
    return all_files

### Process song_data to extract songs & artists dimensional tables
- In this notebook, for the sake of demo, I'll perform ETL on a single song file and load a single record into each table to start.
- I'll use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- I'll select the first song in that list
- I'll read the song file and view the data

In [ ]:
song_files=get_files("data/song_data")

In [ ]:
filepath=song_files[0]

In [ ]:
df=pd.read_json(filepath, lines=True)

In [ ]:
df

#### Songs Table 

Select the columns needed to populate songs table

In [ ]:
songs_df=df[["song_id", "title", "artist_id", "year", "duration"]].values[0]

In [ ]:
songs_data=songs_df.tolist()

In [ ]:
songs_data

Implement the `song_table_insert` in `sql_queries.py` to insert the above record into `songs` table.
Remember to run create_tables.py before running the cell below to ensure you have created/resetted the `songs` table in `sparkifydb` database

In [ ]:
cur.execute(song_table_insert, songs_data)

#### Artists table

Select the columns needed to populate artists table

In [ ]:
artists_df=df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].values[0]

In [ ]:
artists_data=artists_df.tolist()

In [ ]:
cur.execute(artist_table_insert, artists_data)

In [ ]:
conn.close()